In [1]:
## Importando as libs necessárias
from google.colab import files
import os, io, time

# Criando o diretório inicial: /content/
os.chdir('/content/')
try:
  os.mkdir('Imagens', )
except:
  print('A pasta já existe.')
os.chdir('./Imagens')
os.listdir()
uploaded_images = files.upload()



Saving beagle_196.jpg to beagle_196.jpg
Saving Siamese_216.jpg to Siamese_216.jpg
Saving Siamese_104.jpg to Siamese_104.jpg
Saving Siamese_182.jpg to Siamese_182.jpg
Saving Siamese_135.jpg to Siamese_135.jpg
Saving Siamese_79.jpg to Siamese_79.jpg
Saving Siamese_65.jpg to Siamese_65.jpg
Saving Siamese_43.jpg to Siamese_43.jpg
Saving Persian_133.jpg to Persian_133.jpg
Saving Persian_78.jpg to Persian_78.jpg
Saving chihuahua_196.jpg to chihuahua_196.jpg
Saving chihuahua_73.jpg to chihuahua_73.jpg
Saving chihuahua_139.jpg to chihuahua_139.jpg
Saving chihuahua_14.jpg to chihuahua_14.jpg
Saving beagle_9.jpg to beagle_9.jpg
Saving Siamese_202.jpg to Siamese_202.jpg
Saving Persian_182.jpg to Persian_182.jpg
Saving chihuahua_75.jpg to chihuahua_75.jpg
Saving chihuahua_2.jpg to chihuahua_2.jpg
Saving beagle_116.jpg to beagle_116.jpg
Saving beagle_84.jpg to beagle_84.jpg
Saving beagle_32.jpg to beagle_32.jpg
Saving Persian_265.jpg to Persian_265.jpg
Saving Persian_164.jpg to Persian_164.jpg
Savi

# Aplicando HOG

16,16

In [2]:
## Reduzir as imagens
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.feature import hog

hog_features_16x16 = []
for filename in uploaded_images.keys():
  image = imread(filename)
  image_resized = resize(image, (128,128))
  fd, hog_image = hog(image_resized, orientations=9, pixels_per_cell=(16, 16),
                    cells_per_block=(2, 2), visualize=True, channel_axis=-1)
  hog_features_16x16.append(fd)

APLICAR COM 20,20 TAMBÉM

In [3]:
hog_features_20x20 = []
for filename in uploaded_images.keys():
  image = imread(filename)
  image_resized = resize(image, (128,128))
  fd, hog_image = hog(image_resized, orientations=9, pixels_per_cell=(20, 20),
                    cells_per_block=(2, 2), visualize=True, channel_axis=-1)
  hog_features_20x20.append(fd)

In [4]:
import pandas as pd
## Adicionar as colunas com os valores do hog para cada um
data = {"pathfile": uploaded_images.keys(), "hog_features": hog_features_16x16}
df_16x16 = pd.DataFrame(data)
df_16x16['classe'] = df_16x16['pathfile'].str.split('_').str[0]
df_16x16 = df_16x16.drop('pathfile', axis=1)
df_16x16.head()

,hog_features,classe
0,"[0.09709541929124349, 0.012138929340813027, 0....",beagle
1,"[0.37547131717298254, 0.0, 0.0, 0.0, 0.0087113...",Siamese
2,"[0.052850249614843844, 0.03109435323291599, 0....",Siamese
3,"[0.30884693935906404, 0.2581638093634417, 0.11...",Siamese
4,"[0.26270146106207803, 0.26270146106207803, 0.2...",Siamese


In [5]:
data = {"pathfile": uploaded_images.keys(), "hog_features": hog_features_20x20}
df_20x20= pd.DataFrame(data)
df_20x20['classe'] = df_20x20['pathfile'].str.split('_').str[0]
df_20x20 = df_20x20.drop('pathfile', axis=1)
df_20x20.head()

,hog_features,classe
0,"[0.05611603197556734, 0.008790525370453814, 0....",beagle
1,"[0.3552402302689341, 0.0, 0.000840735894222061...",Siamese
2,"[0.030589293744868594, 0.021454182615195154, 0...",Siamese
3,"[0.2844409881440555, 0.20989203972919449, 0.09...",Siamese
4,"[0.28839908613837295, 0.1757875666153769, 0.15...",Siamese


In [6]:
# transformando cada valor do hog_feature em um atributo (coluna)
df2 = pd.DataFrame(df_16x16['hog_features'].tolist())
df2.columns = df2.columns.map(lambda x: f'hog_feature_{x+1}')
df_16x16 = pd.concat([df_16x16.drop('hog_features', axis=1), df2], axis=1)

# Salvando csv resultante
df_16x16.to_csv('HOG_Transform_16x16.csv', sep=';', index=False)

In [7]:
# transformando cada valor do hog_feature em um atributo (coluna)
df2 = pd.DataFrame(df_20x20['hog_features'].tolist())
df2.columns = df2.columns.map(lambda x: f'hog_feature_{x+1}')
df_20x20 = pd.concat([df_20x20.drop('hog_features', axis=1), df2], axis=1)

# Salvando csv resultante
df_20x20.to_csv('HOG_Transform_20x20.csv', sep=';', index=False)

In [8]:
df_16x16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Columns: 1765 entries, classe to hog_feature_1764
dtypes: float64(1764), object(1)
memory usage: 10.8+ MB


In [9]:
df_20x20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Columns: 901 entries, classe to hog_feature_900
dtypes: float64(900), object(1)
memory usage: 5.5+ MB


# PCA com 90%



## 16x16

In [10]:
y_16x16 = df_16x16.classe # Target variable (classe)
X_16x16 = df_16x16.copy().drop('classe', axis=1) # Features

print(X_16x16)
print(y_16x16)

     hog_feature_1  hog_feature_2  hog_feature_3  hog_feature_4  \
0         0.097095       0.012139       0.018061       0.042321   
1         0.375471       0.000000       0.000000       0.000000   
2         0.052850       0.031094       0.044014       0.040870   
3         0.308847       0.258164       0.112411       0.169296   
4         0.262701       0.262701       0.201764       0.107043   
..             ...            ...            ...            ...   
795       0.010492       0.004542       0.009460       0.058155   
796       0.389452       0.074125       0.047025       0.035143   
797       0.023848       0.003091       0.002928       0.004558   
798       0.148467       0.157559       0.209120       0.194960   
799       0.210960       0.017515       0.034871       0.026883   

     hog_feature_5  hog_feature_6  hog_feature_7  hog_feature_8  \
0         0.053472       0.014572       0.006811       0.007677   
1         0.008711       0.000000       0.000000       0.0037

In [11]:
### Importando PCA do SkLearn
from sklearn.decomposition import PCA

##pca_obj = PCA(n_components=4) #para dizer quantos componentes quero ter
pca_obj = PCA(n_components=0.90, whiten=True) #qual porcentagem de informação eu quero
pca_result = pca_obj.fit_transform(X_16x16)

print(pca_result.shape)

colunas = []
for i in range(pca_result.shape[1]):
  colunas.append('pca_'+str(i))

pca_dataset = pd.DataFrame(data = pca_result, columns = colunas) # Atual


(800, 228)


In [12]:
## Juntando o atributo classe
final_data = pca_dataset.join(y_16x16)

# Visualização dos dados normalizados
final_data.head()

# Salvando Pessoa.csv transformado
df_16x16_pca90 = pd.DataFrame(final_data)
df_16x16_pca90.to_csv('16x16_pca-0.90.csv', index=False)

# Importando arquivo transformado
from google.colab import files
files.download('16x16_pca-0.90.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 20x20

In [13]:
y_20x20 = df_20x20.classe # Target variable (classe)
X_20x20 = df_20x20.copy().drop('classe', axis=1) # Features

print(X_20x20)
print(y_20x20)

     hog_feature_1  hog_feature_2  hog_feature_3  hog_feature_4  \
0         0.056116       0.008791       0.012691       0.043213   
1         0.355240       0.000000       0.000841       0.000000   
2         0.030589       0.021454       0.027094       0.029882   
3         0.284441       0.209892       0.094011       0.120177   
4         0.288399       0.175788       0.151478       0.082098   
..             ...            ...            ...            ...   
795       0.016718       0.010317       0.022364       0.069056   
796       0.341304       0.071161       0.042685       0.043462   
797       0.056270       0.059257       0.053730       0.171800   
798       0.173374       0.167597       0.219126       0.219126   
799       0.180278       0.020370       0.029995       0.021034   

     hog_feature_5  hog_feature_6  hog_feature_7  hog_feature_8  \
0         0.054513       0.015406       0.008163       0.010724   
1         0.007180       0.000000       0.000000       0.0027

In [26]:
### Importando PCA do SkLearn
from sklearn.decomposition import PCA

##pca_obj = PCA(n_components=4) #para dizer quantos componentes quero ter
pca_obj = PCA(n_components=0.90, whiten=True) #qual porcentagem de informação eu quero
pca_result = pca_obj.fit_transform(X_20x20)

print(pca_result.shape)

colunas = []
for i in range(pca_result.shape[1]):
  colunas.append('pca_'+str(i))

pca_dataset = pd.DataFrame(data = pca_result, columns = colunas) # Atual


(800, 144)


In [27]:
## Juntando o atributo classe
final_data = pca_dataset.join(y_20x20)

# Visualização dos dados normalizados
final_data.head()

# Salvando Pessoa.csv transformado
df_20x20_pca90 = pd.DataFrame(final_data)
df_20x20_pca90.to_csv('20x20_pca-0.90.csv', index=False)

# Importando arquivo transformado
from google.colab import files
files.download('20x20_pca-0.90.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# PCA com 75%



## 16x16

In [28]:
y_16x16 = df_16x16.classe # Target variable (classe)
X_16x16 = df_16x16.copy().drop('classe', axis=1) # Features

print(X_16x16)
print(y_16x16)

     hog_feature_1  hog_feature_2  hog_feature_3  hog_feature_4  \
0         0.097095       0.012139       0.018061       0.042321   
1         0.375471       0.000000       0.000000       0.000000   
2         0.052850       0.031094       0.044014       0.040870   
3         0.308847       0.258164       0.112411       0.169296   
4         0.262701       0.262701       0.201764       0.107043   
..             ...            ...            ...            ...   
795       0.010492       0.004542       0.009460       0.058155   
796       0.389452       0.074125       0.047025       0.035143   
797       0.023848       0.003091       0.002928       0.004558   
798       0.148467       0.157559       0.209120       0.194960   
799       0.210960       0.017515       0.034871       0.026883   

     hog_feature_5  hog_feature_6  hog_feature_7  hog_feature_8  \
0         0.053472       0.014572       0.006811       0.007677   
1         0.008711       0.000000       0.000000       0.0037

In [29]:
### Importando PCA do SkLearn
from sklearn.decomposition import PCA

##pca_obj = PCA(n_components=4) #para dizer quantos componentes quero ter
pca_obj = PCA(n_components=0.75, whiten=True) #qual porcentagem de informação eu quero
pca_result = pca_obj.fit_transform(X_16x16)

print(pca_result.shape)

colunas = []
for i in range(pca_result.shape[1]):
  colunas.append('pca_'+str(i))

pca_dataset = pd.DataFrame(data = pca_result, columns = colunas) # Atual


(800, 114)


In [30]:
## Juntando o atributo classe
final_data = pca_dataset.join(y_16x16)

# Visualização dos dados normalizados
final_data.head()

# Salvando Pessoa.csv transformado
df_16x16_pca75 = pd.DataFrame(final_data)
df_16x16_pca75.to_csv('16x16_pca-0.75.csv', index=False)

# Importando arquivo transformado
from google.colab import files
files.download('16x16_pca-0.75.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 20x20

In [31]:
y_20x20 = df_20x20.classe # Target variable (classe)
X_20x20 = df_20x20.copy().drop('classe', axis=1) # Features

print(X_20x20)
print(y_20x20)

     hog_feature_1  hog_feature_2  hog_feature_3  hog_feature_4  \
0         0.056116       0.008791       0.012691       0.043213   
1         0.355240       0.000000       0.000841       0.000000   
2         0.030589       0.021454       0.027094       0.029882   
3         0.284441       0.209892       0.094011       0.120177   
4         0.288399       0.175788       0.151478       0.082098   
..             ...            ...            ...            ...   
795       0.016718       0.010317       0.022364       0.069056   
796       0.341304       0.071161       0.042685       0.043462   
797       0.056270       0.059257       0.053730       0.171800   
798       0.173374       0.167597       0.219126       0.219126   
799       0.180278       0.020370       0.029995       0.021034   

     hog_feature_5  hog_feature_6  hog_feature_7  hog_feature_8  \
0         0.054513       0.015406       0.008163       0.010724   
1         0.007180       0.000000       0.000000       0.0027

In [37]:
### Importando PCA do SkLearn
from sklearn.decomposition import PCA

##pca_obj = PCA(n_components=4) #para dizer quantos componentes quero ter
pca_obj = PCA(n_components=0.75, whiten=True) #qual porcentagem de informação eu quero
pca_result = pca_obj.fit_transform(X_20x20)

print(pca_result.shape)

colunas = []
for i in range(pca_result.shape[1]):
  colunas.append('pca_'+str(i))

pca_dataset = pd.DataFrame(data = pca_result, columns = colunas) # Atual


(800, 71)


In [38]:
## Juntando o atributo classe
final_data = pca_dataset.join(y_20x20)

# Visualização dos dados normalizados
final_data.head()

# Salvando Pessoa.csv transformado
df_20x20_pca75 = pd.DataFrame(final_data)
df_20x20_pca75.to_csv('20x20_pca-0.75.csv', index=False)

# Importando arquivo transformado
from google.colab import files
files.download('20x20_pca-0.75.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# TREINAMENTO


Ainda não alterado

## 16x16

### PCA 0.90

In [33]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import confusion_matrix

### Importing Dataset
data = pd.read_csv('16x16_pca-0.90.csv',encoding='utf-8')

y = data.classe # Target variable
X = data.copy().drop('classe', axis=1) # Features

# Split dataset into training set and test set
X_train_70, X_test_30, y_train_70, y_test_30 = train_test_split(X, y, test_size=0.3, random_state=73) # 70% training and 30% test

# Create KNeighborsClassifier object
knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(X_train_70, y_train_70)

#Predict the response for test dataset
y_pred = knn.predict(X_test_30)

# Model Accuracy, how often is the classifier correct?
acuracia = metrics.accuracy_score(y_test_30, y_pred)
print('Accuracy: %.3f' % acuracia)

# Matriz de confusão p/ 30%
confusion_matrix(y_test_30, y_pred)

Accuracy: 0.221


array([[ 3,  2, 52,  4],
       [ 0,  3, 55,  5],
       [ 0,  2, 43,  5],
       [ 0,  9, 53,  4]])

### PCA 0.75

In [34]:
### Importing Dataset
data = pd.read_csv('16x16_pca-0.75.csv',encoding='utf-8')

y = data.classe # Target variable
X = data.copy().drop('classe', axis=1) # Features

# Split dataset into training set and test set
X_train_70, X_test_30, y_train_70, y_test_30 = train_test_split(X, y, test_size=0.3, random_state=73) # 70% training and 30% test

# Create KNeighborsClassifier object
knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(X_train_70, y_train_70)

#Predict the response for test dataset
y_pred = knn.predict(X_test_30)

# Model Accuracy, how often is the classifier correct?
acuracia = metrics.accuracy_score(y_test_30, y_pred)
print('Accuracy: %.3f' % acuracia)

# Matriz de confusão p/ 30%
confusion_matrix(y_test_30, y_pred)

Accuracy: 0.246


array([[ 6,  5, 47,  3],
       [ 2,  4, 52,  5],
       [ 4,  4, 34,  8],
       [ 9,  6, 36, 15]])

## 20X20

### PCA 0.90

In [35]:
### Importing Dataset
data = pd.read_csv('20x20_pca-0.90.csv',encoding='utf-8')

y = data.classe # Target variable
X = data.copy().drop('classe', axis=1) # Features

# Split dataset into training set and test set
X_train_70, X_test_30, y_train_70, y_test_30 = train_test_split(X, y, test_size=0.3, random_state=73) # 70% training and 30% test

# Create KNeighborsClassifier object
knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(X_train_70, y_train_70)

#Predict the response for test dataset
y_pred = knn.predict(X_test_30)

# Model Accuracy, how often is the classifier correct?
acuracia = metrics.accuracy_score(y_test_30, y_pred)
print('Accuracy: %.3f' % acuracia)

# Matriz de confusão p/ 30%
confusion_matrix(y_test_30, y_pred)

Accuracy: 0.258


array([[ 3, 10, 46,  2],
       [ 2,  9, 46,  6],
       [ 0,  6, 38,  6],
       [ 2, 10, 42, 12]])

### PCA 0.75

In [39]:
### Importing Dataset
data = pd.read_csv('20x20_pca-0.75.csv',encoding='utf-8')

y = data.classe # Target variable
X = data.copy().drop('classe', axis=1) # Features

# Split dataset into training set and test set
X_train_70, X_test_30, y_train_70, y_test_30 = train_test_split(X, y, test_size=0.3, random_state=73) # 70% training and 30% test

# Create KNeighborsClassifier object
knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(X_train_70, y_train_70)

#Predict the response for test dataset
y_pred = knn.predict(X_test_30)

# Model Accuracy, how often is the classifier correct?
acuracia = metrics.accuracy_score(y_test_30, y_pred)
print('Accuracy: %.3f' % acuracia)

# Matriz de confusão p/ 30%
confusion_matrix(y_test_30, y_pred)

Accuracy: 0.317


array([[22,  7, 19, 13],
       [ 8, 19, 25, 11],
       [ 3, 12, 22, 13],
       [ 5, 14, 34, 13]])

# Análise de resultados

Foi visto que a acurácia aumentou conforme diminiu a porcentagem de PCA. Também foi visto que a acurácia foi melhor com o HOG 20x20 do que com o HOG 16x16. Logo, os próximos testes serão feitos com HOG 20x20 e PCA baixo, como 50% e 25%.

# Nova aplicação de PCA com HOG 20x20

## PCA

### PCA 0.50

In [40]:
### Importando PCA do SkLearn
from sklearn.decomposition import PCA

##pca_obj = PCA(n_components=4) #para dizer quantos componentes quero ter
pca_obj = PCA(n_components=0.50, whiten=True) #qual porcentagem de informação eu quero
pca_result = pca_obj.fit_transform(X_20x20)

print(pca_result.shape)

colunas = []
for i in range(pca_result.shape[1]):
  colunas.append('pca_'+str(i))

pca_dataset = pd.DataFrame(data = pca_result, columns = colunas) # Atual


(800, 26)


In [41]:
## Juntando o atributo classe
final_data = pca_dataset.join(y_20x20)

# Visualização dos dados normalizados
final_data.head()

# Salvando Pessoa.csv transformado
df_20x20_pca90 = pd.DataFrame(final_data)
df_20x20_pca90.to_csv('20x20_pca-0.50.csv', index=False)

# Importando arquivo transformado
from google.colab import files
files.download('20x20_pca-0.50.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### PCA 0.25

In [42]:
### Importando PCA do SkLearn
from sklearn.decomposition import PCA

##pca_obj = PCA(n_components=4) #para dizer quantos componentes quero ter
pca_obj = PCA(n_components=0.25, whiten=True) #qual porcentagem de informação eu quero
pca_result = pca_obj.fit_transform(X_20x20)

print(pca_result.shape)

colunas = []
for i in range(pca_result.shape[1]):
  colunas.append('pca_'+str(i))

pca_dataset = pd.DataFrame(data = pca_result, columns = colunas) # Atual


(800, 8)


In [43]:
## Juntando o atributo classe
final_data = pca_dataset.join(y_20x20)

# Visualização dos dados normalizados
final_data.head()

# Salvando Pessoa.csv transformado
df_20x20_pca90 = pd.DataFrame(final_data)
df_20x20_pca90.to_csv('20x20_pca-0.25.csv', index=False)

# Importando arquivo transformado
from google.colab import files
files.download('20x20_pca-0.25.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Treinamento 2

### PCA 0.50

In [44]:
### Importing Dataset
data = pd.read_csv('20x20_pca-0.50.csv',encoding='utf-8')

y = data.classe # Target variable
X = data.copy().drop('classe', axis=1) # Features

# Split dataset into training set and test set
X_train_70, X_test_30, y_train_70, y_test_30 = train_test_split(X, y, test_size=0.3, random_state=73) # 70% training and 30% test

# Create KNeighborsClassifier object
knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(X_train_70, y_train_70)

#Predict the response for test dataset
y_pred = knn.predict(X_test_30)

# Model Accuracy, how often is the classifier correct?
acuracia = metrics.accuracy_score(y_test_30, y_pred)
print('Accuracy: %.3f' % acuracia)

# Matriz de confusão p/ 30%
confusion_matrix(y_test_30, y_pred)

Accuracy: 0.421


array([[40,  5, 11,  5],
       [14, 15, 15, 19],
       [ 9,  8, 26,  7],
       [12, 17, 17, 20]])

### PCA 0.25

In [45]:
### Importing Dataset
data = pd.read_csv('20x20_pca-0.25.csv',encoding='utf-8')

y = data.classe # Target variable
X = data.copy().drop('classe', axis=1) # Features

# Split dataset into training set and test set
X_train_70, X_test_30, y_train_70, y_test_30 = train_test_split(X, y, test_size=0.3, random_state=73) # 70% training and 30% test

# Create KNeighborsClassifier object
knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(X_train_70, y_train_70)

#Predict the response for test dataset
y_pred = knn.predict(X_test_30)

# Model Accuracy, how often is the classifier correct?
acuracia = metrics.accuracy_score(y_test_30, y_pred)
print('Accuracy: %.3f' % acuracia)

# Matriz de confusão p/ 30%
confusion_matrix(y_test_30, y_pred)

Accuracy: 0.375


array([[26, 17,  9,  9],
       [ 6, 23, 14, 20],
       [ 8,  9, 19, 14],
       [15, 16, 13, 22]])

# Análise da fase de treinamento 2

Foi percebido que com PCA de 50% foi obitido o maior valor de acurácia. Os próximos testes podem ser feitos com HOG maior que 20x20 e com PCA pouco maior e pouco menor que 50%